# <a id='toc1_'></a>[Salvando clientes de Beta Bank](#toc0_)
  
Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.
  
Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
  
Determinaremos que tan buen modelo tenemos en las manos utilizando métricicas como el puntaje F1 y el AUC-ROC.

**Table of contents**<a id='toc0_'></a>    
- [Salvando clientes de Beta Bank](#toc1_)    
  - [Inicialización](#toc1_1_)    
  - [Carga de datos](#toc1_2_)    
  - [Exploración y carga inicial de datos](#toc1_3_)    
    - [Análisis y reemplazo de valores ausentes en `tenure`](#toc1_3_1_)    
    - [Preparacion de datos para machine learning](#toc1_3_2_)    
  - [Evaluación de balance de clases](#toc1_4_)    
    - [Sobremuestreo](#toc1_4_1_)    
    - [Submuestreo](#toc1_4_2_)    
  - [Segmentación de datos](#toc1_5_)    
  - [Prueba de modelos e hiperparámetros](#toc1_6_)    
    - [Modelo de bosque aleatorio](#toc1_6_1_)    
      - [Conjunto de datos - Sobremuestreo](#toc1_6_1_1_)    
      - [Conjunto de datos - Submuestreo](#toc1_6_1_2_)    
      - [Conjunto de datos - Desbalance de clases](#toc1_6_1_3_)    
    - [Modelo de regresión logística](#toc1_6_2_)    
      - [Conjunto de datos - Sobremuestreo](#toc1_6_2_1_)    
      - [Conjunto de datos - Submuestreo](#toc1_6_2_2_)    
      - [Conjunto de datos - Desbalance](#toc1_6_2_3_)    
  - [Prueba final](#toc1_7_)    
  - [Conclusión](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Inicialización](#toc0_)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## <a id='toc1_2_'></a>[Carga de datos](#toc0_)

In [4]:
data = pd.read_csv('../datasets/Churn.csv')

## <a id='toc1_3_'></a>[Exploración y carga inicial de datos](#toc0_)

In [3]:
# Primero vamos a ver que nos dice info y un panorama de como se ve el DataFrame
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


De lo que podemos observar, las columnas que tenemos son:
- **RowNumber**: índice de cadena de datos
- **CustomerId**: identificador de cliente único
- **Surname**: apellido
- **CreditScore**: valor de crédito
- **Geography**: país de residencia
- **Gender**: sexo
- **Age**: edad
- **Tenure**: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- **Balance**: saldo de la cuenta
- **NumOfProducts**: número de productos bancarios utilizados por el cliente
- **HasCrCard**: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- **IsActiveMember**: actividad del cliente (1 - sí; 0 - no)
- **EstimatedSalary**: salario estimado
  
<ins>**Objetivo**</ins>
- **Exited**: El cliente se ha ido (1 - sí; 0 - no)
  
Lo primero que noto de la tabla es que vamos a tener que renombrar las columnas, y para mi tristeza tendrá que ser manualmente. De ahí podemos observar que la única columna con valores nulos es `Tenure` por la que tendremos que indagar un poco más adelante.
  
Ahora, con el modelo en mente, puedo observar como hay 3 columnas que no nos van a ser muy útiles para entrenar nuestro modelo. Esas son:
1. RowNumber
2. CustomerID
3. Surname
  
Eso se debe a que esas columnas son únicas para cada cliente y no podremos obtener ninguna conclusión con los valores obtenidos de esas celdas.

In [4]:
# Antes de seguir mucho más, quiero corregir las columnas
data = data.rename(
    columns={
        'RowNumber':'row_number',
        'CustomerId':'customer_id',
        'Surname':'surname',
        'CreditScore':'credit_score',
        'Geography':'geography',
        'Gender':'gender',
        'Age':'age',
        'Tenure':'tenure',
        'Balance':'balance',
        'NumOfProducts':'num_of_products',
        'HasCrCard':'has_cr_card',
        'IsActiveMember':'is_active',
        'EstimatedSalary':'estimated_salary',
        'Exited':'exited'
})

Tuve que hacerlo manualmente ya que algunas columnas se les tenia que agregar los _ 
  
Podría haber modificado manualmente los que necesitaban _ y aplicar lower al resto? Si, pero se me acaba de ocurrir esa idea.

In [5]:
# Antes de seguir observemos si hay duplicados en nuestra tabla
data.duplicated().sum()

0

In [6]:
# Tambien veamos que nos dice el describe de la tabla
data.describe()

,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Quise hacer ésto ya que nos permite evaluar la distribución de los datos numéricos de manera más rápida. Tras un par de cálculos mentales podemos notar que la única columna que presenta outliers (valores alejados más de 3 desviaciones estándar de la medaiana) es `num_of_products` que... podemos permitirle ya que por más que se vaya de lo normal, es un factor importante a tener para el banco.
  
Pero a fin de cuentas, éste describe me da tranquilidad ya que no voy a tener que eliminar valores outliers.

In [7]:
# Ahora observemos bien como se distribuyen los datos
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

El hecho de que sean 10000 filas nos permite saber los porcentajes fácilmente y ,aunque desconozco precisamente el margen de tiempo que cubren los datos, que se vayan wl 20% de los clientes ciertamente no es una buena señal. Veamos un poco las columnas `geography`, `num_of_products` y `estimated_salary` en contraste con los exited y no.

In [8]:
# Primero separemos los que se fueron en una variable para compararlos con todo el df
exited = data.loc[data.exited == 1]

In [9]:
# Ahora veamos rápido la comparativa
pd.concat(
    [exited['geography'].value_counts(normalize=True),
     data['geography'].value_counts(normalize=True)],
    axis= 1
)

,geography,geography
Germany,0.399607,0.2509
France,0.397644,0.5014
Spain,0.202749,0.2477


A primera vista notamos como los clientes alemanes parecen más propensos a dejar el banco, mientras que Francia y España no se encuentran muy diferentes entre sí. Acaso los beneficios que da el banco no son los mejores en el país de las cervezas?

In [10]:
# Hay algún detalle similar en num_of_products?
pd.concat(
    [
        exited['num_of_products'].value_counts(normalize=True),
         data['num_of_products'].value_counts(normalize=True)
    ],
    axis= 1
)

,num_of_products,num_of_products
1,0.691703,0.5084
2,0.170839,0.4590
3,0.108002,0.0266
4,0.029455,0.0060


A pesar de que la mayoria de los clientes tiene entre 1 y 2 productos del banco, el grueso de los que se van del banco tenían 1 producto, capaz que si se les ofrece un beneficio inicial para algún otro producto se logre un gran paso para que se queden!

In [11]:
# Finalmente veamos estimated_salary
pd.concat(
    [
        exited['estimated_salary'].describe(),
        data['estimated_salary'].describe()
    ],
    axis= 1
)

,estimated_salary,estimated_salary
count,2037.000000,10000.000000
mean,101465.677531,100090.239881
std,57912.418071,57510.492818
min,11.580000,11.580000
25%,51907.720000,51002.110000
50%,102460.840000,100193.915000
75%,152422.910000,149388.247500
max,199808.100000,199992.480000


Acá vemos que tienen patrones muy similares entre sí por lo que podemos confiar en que el sueldo de una persona no influencia en si se va a ir o no del banco, lo que es una buena señal!

Por lo tanto, a primera vista podemos concluir que efectivamente la situación del banco es una que merece ser destacada y tratada con urgencia ya que 2000 clientes no es poca cosa. Los factores que llevan a que un cliente se vaya parecen depender en parte de su país de residencia y de la cantidad de productos que tengan pero no de cuanto ganan. Se podrían estudiar todas las columnas y realizar un análisis mucho más profundo al respecto pero no es el enfoque que vamos a tratar hoy. Hoy vamos a buscar hacer el modelo perfecto!... O acercarnos lo suficiente.

### <a id='toc1_3_1_'></a>[Análisis y reemplazo de valores ausentes en `tenure`](#toc0_)

In [12]:
# Primero guardemos en una variable las filas que les falta tenure y en otra las que no
missing_tenure = data.loc[data.tenure.isna()]

not_missing_tenure = data.loc[~data.tenure.isna()]

In [13]:
# Veamos entonces las filas con ausentes
missing_tenure

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [14]:
not_missing_tenure

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,15719294,Wood,800,France,Female,29,2.0,0.00,2,0,0,167773.55,0
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1


In [15]:
for column in data.columns.values:
    if column not in ['row_number','customer_id','surname','tenure']:
        print(pd.concat(
        [
            missing_tenure.value_counts(column),
            not_missing_tenure.value_counts(column)
        ], axis=1
        ),'\n')

                 0      1
credit_score             
350            NaN    5.0
351            NaN    1.0
358            NaN    1.0
359            1.0    NaN
363            NaN    1.0
...            ...    ...
846            1.0    4.0
847            NaN    6.0
848            NaN    5.0
849            NaN    8.0
850           23.0  210.0

[460 rows x 2 columns] 

             0     1
geography           
France     464  4550
Spain      229  2248
Germany    216  2293 

          0     1
gender           
Male    483  4974
Female  426  4117 

       0   1
age         
18   2.0  20
19   1.0  26
20   3.0  37
21   5.0  48
22   4.0  80
..   ...  ..
83   NaN   1
84   NaN   2
85   NaN   1
88   NaN   1
92   1.0   1

[70 rows x 2 columns] 

               0       1
balance                 
0.00       334.0  3283.0
3768.69      NaN     1.0
12459.19     NaN     1.0
14262.80     NaN     1.0
16893.59     1.0     NaN
...          ...     ...
216109.88    NaN     1.0
221532.80    NaN     1.0
222267.63  

Para nuestra mala suerte, no hay un patrón muy evidente respecto a la causa de los ausentes por lo que vamos a tener que buscar la forma de imputar el valor.

In [16]:
# Primero observemos la distribución de tenure
not_missing_tenure['tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: tenure, dtype: float64

In [17]:
# Viendo que la mediana y la media son iguales, vamos a reemplazar los valores ausentes con 5
filled_data = data.fillna(5)

# Y tambien vamos a guardar una tabla aparte en donde solo vamos a tener valores puros, para comparar
clean_data = not_missing_tenure

In [18]:
# Finalmente verificamos el impacto del cambio en nuestros datos
data['tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: tenure, dtype: float64

Dado que el cambio que hicimos no tuvo un impacto notorio, vamos a mantener éste cambio y proseguir con el proyecto. Posteriormente haremos comparaciones tambien para ver si es mejor el dataset limpio o con los valores imputados.

### <a id='toc1_3_2_'></a>[Preparacion de datos para machine learning](#toc0_)
  
Ahora nos dedicaremos a procesar los datos de tal forma que queden aptos para entrenar a un modelo. Eso incluirá principalmente transformar las columnas categóricas en multiples columnas OHE (One-Hot Encoding).
  
Para eso, primero observemos una vez más la tabla.

In [19]:
data

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Al parecer lo que nos toca ahora es simplemente transformar las columnas `geography` y `gender` en formato OHE ya que las columnas que podriamos calificar como Ordinal ya estan de esa forma. Para nuestra suerte, pandas tiene una función que nos permite hacer éste cambio muy fácil: `pd.get_dummies()`

In [20]:
# Definimos las columnas que vamos a transformar
dummy_columns = ['geography', 'gender']

# Pero tenemos que aclarar el parámetro drop_first para no caer en la trampa dummy
filled_data = pd.get_dummies(filled_data, columns= dummy_columns, drop_first= True)

In [21]:
# Y no nos olvidemos de la tabla limpia
clean_data = pd.get_dummies(clean_data, columns= dummy_columns, drop_first= True)

## <a id='toc1_4_'></a>[Evaluación de balance de clases](#toc0_)
  
Ahora nos vamos a centrar en ver si las diferentes clases estan muy desbalanceadas o no. Para nuestra suerte, podemos generalizar el trabajo tanto para `filled_data` como para `clean_data` ya que vimos que presentan distribuciones muy similares.

In [22]:
# Lo primero que vamos a hacer es dterminar las columnas que no nos sirven para entrenar
useless_columns = ['row_number','customer_id','surname']

# De ahi guardamos en variables nuevas solo con datos para entrenar
useful_filled_data = filled_data.drop(columns= useless_columns)
useful_clean_data = clean_data.drop(columns= useless_columns)

In [23]:
useful_filled_data

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


In [24]:
# Primero veamos la distribucion de clases de exited en useful_clean_data
useful_clean_data['exited'].value_counts(normalize= True)

0    0.796062
1    0.203938
Name: exited, dtype: float64

In [25]:
# Veamos entonces la distribución de clases de exited
useful_filled_data['exited'].value_counts(normalize= True)

0    0.7963
1    0.2037
Name: exited, dtype: float64

Claramente eso no es una clase balanceada. Lo que vamos a hacer ahora es trabajar con las tablas de tal manera que para el final de ésta etapa terminemos con 3 variables importantes:
- La tabla de datos tras un proceso de sobremuestreo
- La tabla de datos tras un proceso de submuestreo
- La tabla de datos sin procesar el desbalance de clases
  
Posteriormente evaluaremos cual de las tablas tiene un mejor rendimiento para el modelo que elijamos y con esa información conocida haremos el balance de clases usado para la tabla de `useful_clean_data`

### <a id='toc1_4_1_'></a>[Sobremuestreo](#toc0_)
  
Para efectuar el sobremuestreo primero vamos a crear una función que lo haga por nosotros. Aunque antes de eso, debemos separar los datos en *features y target*.

In [26]:
# Guardamos las features
filled_features = useful_filled_data.drop(columns= ['exited'])
filled_target = useful_filled_data['exited']

In [27]:
def upsample(features, target, repeat):
    
    # Primero separamos los 0 y 1 de target
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # Posteriormente aumentamos la cantidad de 0 en los datos
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # Finalmente mezclamos todo
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=46587)
    
    # Y devolvemos las variables 
    return features_upsampled, target_upsampled


In [28]:
# Obtengamos entonces los datos sobremuestreados (eso es una palabra?)
upsamp_ffeatures, upsamp_ftarget = upsample(filled_features, filled_target, 4)

In [29]:
# Y veamos como quedó el balance
print(upsamp_ffeatures.shape)
upsamp_ftarget.value_counts(normalize= True)

(16111, 11)


1    0.505741
0    0.494259
Name: exited, dtype: float64

Vemos que logramos balancear las clases bastante bien y aumentamos las filas totales en 6111 lo cual no es poco, pero el desbalance presentado lo requería.
  
### <a id='toc1_4_2_'></a>[Submuestreo](#toc0_)
  
Ahora vamos a seguir un procedimiento muy similar al que tuvimos previamente pero al revés. Mi predicción es que éste balance nos hará perder una buena porción de los datos pero vamos a ello.

In [30]:
def downsample(features, target, fraction):
    # Separamos una vez más los datos en 0 y 1
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # Para reducir los datos tenemos que llamar a una funcion de pandas
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=54321)] + [features_ones])
    
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=54321)] + [target_ones])
    
    # Finalmente mezclamos los datos
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=54321)
    
    # Y devolemos las tablas ya submuestreadas
    return features_downsampled, target_downsampled


In [31]:
# Obtenemos los datos tras submuestrearlos (otra palabra que desconozco su existencia)
downsamp_ffeatures, downsamp_ftarget = downsample(filled_features, filled_target, 0.256123)

In [32]:
# Y verificamos su distribución
print(downsamp_ffeatures.shape)
downsamp_ftarget.value_counts(normalize= True)

(4077, 11)


0    0.500368
1    0.499632
Name: exited, dtype: float64

La ventaja que tenemos en el downsample es que toma valores flotantes por lo que podemos lograr un balance mucho más preciso aunque jamas lograremos llegar a ese 50/50 deseado. La desventaja es que perdimos casi 6000 datos y como sabemos, mientras más datos tengamos para nuestro modelo, mejor.
  
Por lo tanto, terminamos con éstos tres conjuntos de datos:
- `upsamp_ffeatures` y `upsamp_ftarget`: Los datos tras un balance basado en el sobremuestreo
- `downsamp_ffeatures` y `downsamp_ftarget`: Los datos tras un balance basado en el submuestreo
- `filled_features` y `filled_target`: Los datos desbalanceados
  
En la siguientes etapas nos dedicaremos a segmentar los datos, buscar los mejores modelos para nuestros datos y elegir un conjunto definitivo sobre el cual llevaremos a cabo las pruebas.

## <a id='toc1_5_'></a>[Segmentación de datos](#toc0_)
  
Vamos a usar la función `GridSearchCV` para la optimización de hiperparámetros la que nos permitirá reducir el sobreajuste de nuestros modelos. Para lograr de mejor manera eso, vamos a dividir los datos en 2 partes: entrenamiento (75%) y pruebas (25%).

De no usar `GridSearchCV` si habría que separar el dataset en 3 partes para tener un conjunto de validación lo que como consecuencia reduciría el tamaño total del conjunto de entrenamiento.

In [33]:
# Ahora vamos a separar los grupos de los 3 conjuntos de datos que mecionamos previamente
# Primero los datos sobremuestreados (UPSampled)
ups_ffeatures_train, ups_ffeatures_test, ups_ftarget_train, ups_ftarget_test = train_test_split(
    upsamp_ffeatures, upsamp_ftarget, test_size=0.25, random_state=45695)

# De ahi los submuestreados (DoWnSampled)
dws_ffeatures_train, dws_ffeatures_test, dws_ftarget_train, dws_ftarget_test = train_test_split(
    downsamp_ffeatures, downsamp_ftarget, test_size=0.25, random_state=45695)

# Finalmente los datos desbalanceados (UNBalanced)
unb_ffeatures_train, unb_ffeatures_test, unb_ftarget_train, unb_ftarget_test = train_test_split(
    filled_features, filled_target, test_size=0.25, random_state=45695)

Me parece que antes de seguir voy a aclarar bien la nomenclatura de todas esas variables. Antes de ir a esas variables, veamos de vuelta que tenemos antes:
- `upsamp_ffeatures` y `upsamp_ftarget`: Los datos tras un balance basado en el sobremuestreo
- `downsamp_ffeatures` y `downsamp_ftarget`: Los datos tras un balance basado en el submuestreo
- `filled_features` y `filled_target`: Los datos desbalanceados
- `useful_clean_data`: La tabla sin separar en features o targets a la que se le eliminaron las filas con valores nulos
    
Es por eso que la estructura general de los nombres tiene 3 partes para 4 cosas:
  
`[A]`_`[B]` `[C]`_`[D]`
- **A:** Sobremuestreados (ups) / Submuestreados (dws) / Desbalanceados (unb)
- **B:** Datos imputados (f) / Limpios (c)
- **C:** Features (features) / Target (target)
- **D:** De prueba (train) / De entrenameiento (test)

## <a id='toc1_6_'></a>[Prueba de modelos e hiperparámetros](#toc0_)
  
En éste caso nos encontramos con una problemática de clasificación por lo que vamos a probar 2 modelos:

- Modelo de bosque alteatorio (RandomForestClassifier)
- Modelo de regresión logística (LogisticRegression)
  
Como dijimos, vamos a usar `GridSearchCV` para obtener la mejor combinación de hiperparámetros acorde a nuestros datos y a los modelos en cuestión.
  
Ésta etapa será algo tediosa ya que tendremos que entrenar 3 modelos para determinar cuál es el mejor.

<h1> Disclaimer </h1>
  
**Al hacer el proyecto yo usé la función GridSearchCV pero ésta se toma mucho tiempo por el amplio rango de parámetros que elegí. Ante ésto yo copié el modelo optimizado y lo guardé en la variable. Todo el código original relacionado a la función GridSearchCV está comentado tal y como lo usé en el proyecto con los fines de no tener que esperar tanto para ejecutar el mismo.**

### <a id='toc1_6_1_'></a>[Modelo de bosque aleatorio](#toc0_)
  
Un modelo muy capaz pero con un coste computacional muy alto tambien. Como dijimos, vamos a tener que entrenar 3 modelos diferentes para poder compararlos y ver cuál de los dos métodos de balance que usamos es mejor, si es que alguno es mejor.

#### <a id='toc1_6_1_1_'></a>[Conjunto de datos - Sobremuestreo](#toc0_)

In [34]:
# Primero determinaremos sobre cuales hiperparámetros vamos a trabajar
params = {
    'n_estimators': [50,75,100],
    'criterion': ['entropy'],
    'max_depth': list(range(2,7)),
    'min_samples_split': list(range(2,10)),
    'min_samples_leaf': list(range(1,6)),
    'max_features': ['sqrt', 'log2', None],
    'random_state': [45695]
}

# Cargamos el grid en una variable
forest_grid = GridSearchCV(RandomForestClassifier(), param_grid= params, cv= 5, verbose= 1)

In [35]:
# Primero buscamos el mejor conjunto de h-parametros para los datos sobremuestreados

#forest_grid.fit(ups_ffeatures_train, ups_ftarget_train)

In [36]:
# Veamos cual es el mejor modelo

#forest_grid.best_estimator_

In [37]:
# Y cargemos ese modelo en una variable
#ups_forest_model = forest_grid.best_estimator_

ups_forest_model = RandomForestClassifier(criterion='entropy', max_depth=6, max_features=None,
                       min_samples_split=3, n_estimators=75,
                       random_state=45695)

In [38]:
# De ahi entrenamos el modelo
ups_forest_model.fit(ups_ffeatures_train, ups_ftarget_train)

RandomForestClassifier(criterion='entropy', max_depth=6, max_features=None,
                       min_samples_split=3, n_estimators=75,
                       random_state=45695)

In [39]:
# Finalmente guardamos la predicción
ups_forest_predict = ups_forest_model.predict(ups_ffeatures_test)

In [40]:
# Ahora veamos el puntaje F1 y el puntaje AUC-ROC
print(
f' Valor F1: {f1_score(ups_ftarget_test, ups_forest_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(ups_ftarget_test, ups_forest_predict):.5f}'
)

 Valor F1: 0.78506
 Valor AUC-ROC: 0.77989


Podemos observar que las grid ciertamente lograron una buena combinación de hiperparámetros ya que el puntaje F1 está bastante por encima del umbral requerido. Tambien podemos observar que el valor AUC-ROC es un poco menor que el F1 pero de todas formas es casi 0,8. Al parecer el sobremuestreo junto con los hiperparámetros obtenidos con `GridSearchCV` obtienen un muy buen resultado.
    
Pasemos entonces a ver como se comportan el resto de los datos.

#### <a id='toc1_6_1_2_'></a>[Conjunto de datos - Submuestreo](#toc0_)

In [41]:
# Determinaremos sobre cuales hiperparámetros vamos a trabajar
params = {
    'n_estimators': [50,75,100],
    'criterion': ['entropy'],
    'max_depth': list(range(2,7)),
    'min_samples_split': list(range(2,10)),
    'min_samples_leaf': list(range(1,6)),
    'max_features': ['sqrt', 'log2', None],
    'random_state': [45695]
}

# Cargamos el grid en una variable
forest_grid = GridSearchCV(RandomForestClassifier(), param_grid= params, cv= 5, verbose= 1)

In [42]:
# Primero buscamos el mejor conjunto de h-parametros para los datos sobremuestreados

#forest_grid.fit(dws_ffeatures_train, dws_ftarget_train)

In [43]:
# Veamos cual es el mejor modelo

# forest_grid.best_estimator_

Algo que quiero destacar es como el conjunto de hiperparámetros para éstos datos es difertente al de los sobremuestreados. Aunque tiene sentido ya que tratamos con diferentes datos.
  
Me llama la atención como `max_features` acá es la raiz cuadrada de los totales cuando en el otro modelo no había un límite. También notamos como éste bosque tiene 25 árboles menos. Aunque... el `min_samples_split` es el doble... al parecer éstos árboles tienen ramas "más duras"! 

In [44]:
# Y cargemos ese modelo en una variable
#dws_forest_model = forest_grid.best_estimator_

dws_forest_model = RandomForestClassifier(criterion='entropy', max_depth=6, max_features='sqrt',
                       min_samples_split=6, n_estimators=50,
                       random_state=45695)

In [45]:
# De ahi entrenamos el modelo
dws_forest_model.fit(dws_ffeatures_train, dws_ftarget_train)

RandomForestClassifier(criterion='entropy', max_depth=6, max_features='sqrt',
                       min_samples_split=6, n_estimators=50,
                       random_state=45695)

In [46]:
# Finalmente guardamos la predicción
dws_forest_predict = dws_forest_model.predict(dws_ffeatures_test)

In [47]:
# Ahora veamos el puntaje F1 y el puntaje AUC-ROC
print(
f' Valor F1: {f1_score(dws_ftarget_test, dws_forest_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(dws_ftarget_test, dws_forest_predict):.5f}'
)

 Valor F1: 0.73504
 Valor AUC-ROC: 0.75479


Al parecer el submuestreo no es la mejor de las dos técnicas de balance para nuestro caso, no es que éste sea una mala opción pues ~0.75 no es un valor para despreciar! Y vemos como el modelo logra en promedio un 76,7% verdaderos positivos gracias al puntaje AUC-ROC.

#### <a id='toc1_6_1_3_'></a>[Conjunto de datos - Desbalance de clases](#toc0_)

In [48]:
# Determinaremos sobre cuales hiperparámetros vamos a trabajar
params = {
    'n_estimators': [50,75,100],
    'criterion': ['entropy'],
    'max_depth': list(range(2,7)),
    'min_samples_split': list(range(2,10)),
    'min_samples_leaf': list(range(1,6)),
    'max_features': ['sqrt', 'log2', None],
    'random_state': [45695]
}

# Cargamos el grid en una variable
forest_grid = GridSearchCV(RandomForestClassifier(), param_grid= params, cv= 5, verbose= 1)

In [49]:
# Primero buscamos el mejor conjunto de h-parametros para los datos sobremuestreados

# forest_grid.fit(unb_ffeatures_train, unb_ftarget_train)

In [50]:
# Veamos cual es el mejor modelo

# forest_grid.best_estimator_

In [51]:
# Y cargemos ese modelo en una variable
#unb_forest_model = forest_grid.best_estimator_

unb_forest_model = RandomForestClassifier(criterion='entropy', max_depth=6, max_features=None,
                       min_samples_leaf=5, n_estimators=75, random_state=45695)

In [52]:
# De ahi entrenamos el modelo
unb_forest_model.fit(unb_ffeatures_train, unb_ftarget_train)

RandomForestClassifier(criterion='entropy', max_depth=6, max_features=None,
                       min_samples_leaf=5, n_estimators=75, random_state=45695)

In [53]:
# Finalmente guardamos la predicción
unb_forest_predict = unb_forest_model.predict(unb_ffeatures_test)

In [54]:
# Ahora veamos el puntaje F1 y el puntaje AUC-ROC
print(
f' Valor F1: {f1_score(unb_ftarget_test, unb_forest_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(unb_ftarget_test, unb_forest_predict):.5f}'
)

 Valor F1: 0.55456
 Valor AUC-ROC: 0.70267


Efectivamente vemos porque hay que balancear los datos! Éste modelo ni llega a pasar el umbral que debiamos tratar... Y pensar que ese modelo tiene una muy buena combinación de hiperparámetros! Por otra parte el valor AUC-ROC se ve notoriamente alto, pero eso se debe a que dicho valor no es óptimo para conjuntos de datos inbalanceados.
  
Por lo tanto, el ganador dentro de la categoría es el modelo con los datos balanceados mediante sobremuestreo! Con un valor f1 de 0,78506 y un puntaje AUC-ROC de 0,77989. Ya despues analizaremos bien que puede haber causado la diferencia entre los dos conjuntos de datos balanceados.

### <a id='toc1_6_2_'></a>[Modelo de regresión logística](#toc0_)
  
Un método muy diferente al de árboles de decisión ya que la forma de obtener la respuesta es formando una función matemática y en base a eso determina el resultado para una dada característica.
  
Por supuesto, vamos a usar una vez más el `GridSearchCV` para determinar la mejor combinación de hiperparámetros para nuestros datos.

#### <a id='toc1_6_2_1_'></a>[Conjunto de datos - Sobremuestreo](#toc0_)

In [55]:
# Primero vamos a determinar los h-parámetros que vamos a probar
params = {
    'C': np.logspace(-4,4,35),
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': list(range(100,310,50)),
    'random_state': [45695]
}

logistic_grid = GridSearchCV(LogisticRegression(), param_grid= params, cv= 10, verbose= 1)

In [56]:
# Ahora determinamos los h-parámetros

# logistic_grid.fit(ups_ffeatures_train, ups_ftarget_train)

In [57]:
# Entonces, cual es la mejor combincación?
# logistic_grid.best_estimator_

In [58]:
# Cargamos el modelo en una variable
#ups_logistic_model = logistic_grid.best_estimator_

ups_logistic_model = LogisticRegression(C=0.0015013107289081743, random_state=45695,
                   solver='liblinear')

In [59]:
# Lo entrenamos
ups_logistic_model.fit(ups_ffeatures_train, ups_ftarget_train)

LogisticRegression(C=0.0015013107289081743, random_state=45695,
                   solver='liblinear')

In [60]:
# Guardamos su predicción
ups_logistic_predict = ups_logistic_model.predict(ups_ffeatures_test)

In [61]:
# Y finalmente vemos sus puntajes!
print(
f' Valor F1: {f1_score(ups_ftarget_test, ups_logistic_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(ups_ftarget_test, ups_logistic_predict):.5f}'
)

 Valor F1: 0.67486
 Valor AUC-ROC: 0.66461


Al parecer nuestro modelo logístico no es un mal modelo tampoco! Con un puntaje F1 de 0,67 ciertamente es peor que su equivalente de bosque aleatorio pero si tomamos en cuenta el costo computacional de ambos éste es mejor. Tambien notamos como el puntaje AUC-ROC tambien se encuentra un poco por detras del F1 similar a lo que vimos en el bosque aleatorio.
  
Pasemos entonces a entrenar el modelo con datos submuestreados para comparar como se comporta el modelo logístico frente al modelo de bosques aleatorios.

#### <a id='toc1_6_2_2_'></a>[Conjunto de datos - Submuestreo](#toc0_)

In [62]:
# Primero vamos a determinar los h-parámetros que vamos a probar
params = {
    'C': np.logspace(-4,4,35),
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': list(range(100,310,50)),
    'random_state': [45695]
}

logistic_grid = GridSearchCV(LogisticRegression(), param_grid= params, cv= 10, verbose= 1)

In [63]:
# Ahora determinamos los h-parámetros

#logistic_grid.fit(dws_ffeatures_train, dws_ftarget_train)

In [64]:
# Entonces, cual es la mejor combincación?

# logistic_grid.best_estimator_

In [65]:
# Cargamos el modelo en una variable
#dws_logistic_model = logistic_grid.best_estimator_

dws_logistic_model = LogisticRegression(C=0.11450475699382812, random_state=45695,
                   solver='liblinear')

In [66]:
# Lo entrenamos
dws_logistic_model.fit(dws_ffeatures_train, dws_ftarget_train)

LogisticRegression(C=0.11450475699382812, random_state=45695,
                   solver='liblinear')

In [67]:
# Guardamos su predicción
dws_logistic_predict = dws_logistic_model.predict(dws_ffeatures_test)

In [68]:
# Y finalmente vemos sus puntajes!
print(
f' Valor F1: {f1_score(dws_ftarget_test, dws_logistic_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(dws_ftarget_test, dws_logistic_predict):.5f}'
)

 Valor F1: 0.65121
 Valor AUC-ROC: 0.66072


Al parecer vemos una situación muy similar a la del bosque aleatorio ya que el conjunto sobremuestreado presenta de vuelta tanto un valor F1 como un AUC-ROC un poco mayor al submuestreado, e incluso podemos notar como en los dos pares de casos el conjunto submuestreado presenta un valor AUC-ROC ligeramente superior al F1. 
  
Ésto último nos indica algo respecto al comportamiento de los datos ante el sobremuestreo y el submuestreo. Mi hipótesis es que si bien estamos trabajando con 10000 filas, al parecer reducir la cantidad total de filas para el entrenamiento tiene una ligera desventaja ya que terminamos con menos de la mitad y eso repercute ligeramente sobre la calidad de un modelo dado que tiene menos datos sobre los cuales entrenarse.

#### <a id='toc1_6_2_3_'></a>[Conjunto de datos - Desbalance](#toc0_)

In [69]:
# Primero vamos a determinar los h-parámetros que vamos a probar
params = {
    'C': np.logspace(-4,4,35),
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': list(range(100,310,50)),
    'random_state': [45695]
}

logistic_grid = GridSearchCV(LogisticRegression(), param_grid= params, cv= 10, verbose= 1)

In [70]:
# Ahora determinamos los h-parámetros

# logistic_grid.fit(unb_ffeatures_train, unb_ftarget_train)

In [71]:
# Entonces, cual es la mejor combincación?

# logistic_grid.best_estimator_

In [72]:
# Cargamos el modelo en una variable
#unb_logistic_model = logistic_grid.best_estimator_

unb_logistic_model = LogisticRegression(C=0.0001, random_state=45695, solver='liblinear')

In [73]:
# Lo entrenamos
unb_logistic_model.fit(unb_ffeatures_train, unb_ftarget_train)

LogisticRegression(C=0.0001, random_state=45695, solver='liblinear')

In [74]:
# Guardamos su predicción
unb_logistic_predict = unb_logistic_model.predict(unb_ffeatures_test)

In [75]:
# Y finalmente vemos sus puntajes!
print(
f' Valor F1: {f1_score(unb_ftarget_test, unb_logistic_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(unb_ftarget_test, unb_logistic_predict):.5f}'
)

 Valor F1: 0.10450
 Valor AUC-ROC: 0.52027


Una vez más podemos notar el impacto de entrenar un modelo desbalanceado ya que éste modelo se comporta mucho peor que tirar una moneda!
  
Sin más vueltas, pasemos a la conclusión final sobre cual modelo es el mejor!

## <a id='toc1_7_'></a>[Prueba final](#toc0_)

Antes de pasar en sí a la prueba final, recordemos los puntajes de los diferentes modelos:

| Modelo | Sobremuestreo (F1) | Sobremuestreo (AUC-ROC) | Submuestreo (F1) | Submuestreo (AUC-ROC) |
|----------|----------|----------|----------|----------|
|Bosque Aleatorio|0.78506|0.77989|0.73504|0.75479|
|Regresión Logística|0.67486|0.66461|0.65121|0.66072|
  
Como podemos ver, el modelo de Bosque Aleatorio de Decisiones obtuvo mejores métricas en todos los conjuntos y los conjuntos sobremuestreados obtuvieron mejores resultados en ambos modelos que los submuestreados.
  
En el primer caso es evidente ya que el modelo de Bosque aleatorio es un modelo mucho más complejo que logra adaptarse mucho mejor a las diferentes conexiones que hay entre las features para deducir el resultado mientras que la regresión logística si bien también "aprende" de todas las features, tambien las considera a todas a la vez mediante la fórmula matemáteica que la define. Un detalle no menor, el tiempo de ejecución de la Regresión es mucho menor al de los Bosques Aleatorios y ambos modelos pasan el umbral estipulado al comienzo.
  
Por lo tanto, nosotros nos vamos a quedar con el modelo de Bosque Aleatorio con datos sobremuestreados! Entonces pasemos a la prueba final.

In [76]:
# Primero guardamos el modelo de bosque aleatorio en una variable para no perderlo
final_model = ups_forest_model

In [77]:
# De ahi vamos a mezclar todo el dataset para realizar la prueba final con todo el conjunto, usamos un nuevo random_state
# para intentar simular un conjunto completamente nuevo
final_features, final_target = filled_features, filled_target

In [78]:
# Vemos que predice el modelo
final_predict = final_model.predict(final_features)

In [79]:
# Entonces veamos el puntaje final!!
print(
f' Valor F1: {f1_score(final_target, final_predict):.5f}'
f'\n Valor AUC-ROC: {roc_auc_score(final_target, final_predict):.5f}'
)

 Valor F1: 0.59657
 Valor AUC-ROC: 0.78791


Bueno... esos... no son los resultados que esperaba. Técnicamente superamos el umbral requerido pero no puedo evitar notar la gran diferencia en puntajes que vimos hasta ahora. Lo más probable es que el problema nazca de que el modelo se haya acostumbrado a las "personas imaginarias" que creamos a partir del sobremuestreo y que su "tren de pensamiento" presente un sesgo debido a que hicimos parecer normal algo que no era. Basicamente, el modelo vió que la gran mayoría de los que se iban del banco presentaban las mismas características hasta el más minimo detalle y eso seguro causó que busque patrones que no necesariamente se correlacionan o capaz que hizo una suerte de sobreajuste en la que los márgenes para variables numéricas eran muy chicos y no daban margen para otros casos.

## <a id='toc1_8_'></a>[Conclusión](#toc0_)
  
Después de muchas pruebas, mucho entrenamiento y varios resultados finalmente llegamos a la conclusión! Vamos por partes:
  
1. **Los datos**
  
La tabla que nos dieron para realizar el trabajo estaban en una condición casi perfecta, no hubo que corregir los tipos de las columnas, no presentaban valores duplicados y solo un ~10% presentaban valores ausentes y en solo una columna. No puedo apuntar a un motivo concreto sobre el que pueda justificar la presencia de esos valores ausentes pues sus distribuciones iban en concordancia con el resto del dataframe por lo que creo que es mejor estudiar todo el camino que pasaron esos datos para encontrar la causa más que buscar un patrón en su origen o características.
  
Por otra parte, me parece que la tabla si merece un análisis riguroso ya que nuestra busqueda superficial ya empezó a revelar múltiples factores en común sobre los que vale la pena ahondar e investigar (como que la mayoria de los clientes que se van tienen solo 1 producto). Mi recomendación sería pasar la tabla a un Data Analyst ya que si bien un modelo de Machine Learning puede darnos una respuesta de quién se está por ir, no nos va a dar la solución sobre que debemos hacer para ese caso. En cambio, con un análisis más profundo podriamos descubrir no solo las causas detras de los clientes yendose sino también unas soluciones aptas para cada situación.
  
Finalmente hay que nombrar que antes de poder trabajar con nuestros datos respecto a Machine Learning tuvimos que transformar las columnas `gender` y `geography` con la metodología One-Hot Coding ya que no podíamos aplicar el método ordinal porque no queremos hacerle creer que los países o los géneros poseen una relación en la que alguno es mayor o menor.
  
2. **Desbalance de datos**
 
Si bien los datos se encontraban relativamente limpios, no podemos decir lo mismo de la distribución de los mismos ya que encontramos como el 79% de los mismos no se habían ido del banco. Para eso probamos 2 técnicas diferentes de balance: el sobremuestreo y el submuestreo. Como vimos antes, consiste en repetir/eliminar las filas del conjunto que es más raro/común para así lograr un balance y llevar la proporcion a un ratio 1:1.
  
Honestamente si tienen sus detalles ambos métodos ya que es elegir uno de dos caminos: O repetimos datos y creamos personas imaginarias que coinciden hasta el último detalle con otras, o eliminamos datos que tienen el potencial de proporcionar nuevos conocimientos sobre la relación entre las features y los datos. Ya hablaremos un poco más de eso más adelante.
  
3. **Entrenamiento de modelos**
  
De ahi pasamos al entrenamiento de modelos, entrenamos un total de 6 modelos diferentes de los cuales 3 eran bosque aleatorio y 3 eran de regresión logística. Eran 3 de cada uno ya que quería evaluar los comportamientos frente a de los conjuntos sin balancear. Fue más que evidente que los modelos entrenados sin balancear los datos terminaban mucho peor ya que obtuvieron un puntaje F1 de 0,55456 en el caso de bosque aleatorio y 0,10450 en el caso de la regresión logística. Mientras tanto, el resto de los modelos demostraron unos valores F1 muy buenos:

| Modelo | Sobremuestreo (F1) | Sobremuestreo (AUC-ROC) | Submuestreo (F1) | Submuestreo (AUC-ROC) |
|----------|----------|----------|----------|----------|
|Bosque Aleatorio|0.78506|0.77989|0.73504|0.75479|
|Regresión Logística|0.67486|0.66461|0.65121|0.66072|

Estoy usando de vuelta la tabla? Si, es una buena manera de mostrar los valores y poder comparar lado a lado todo. Como dije previamente, el modelo de bosque aleatorio demostró puntajes F1 superiores al de la regresión logística pero con un tiempo de ejecución mayor tambien. Ese ultimo detalle no es menor ya que de no tener las capacidades computacionales para albergar el modelo y que funcione en tiempos coherentes, puede que sea mejor el modelo logístico. 
  
Otra cosa que hay que destacar es que los modelos fueron evaluados principalemente con el valor F1 que es una medida armónica entre la *precision* y el *recall* por lo que dan una proporción balanceada de errores tanto para los negativos como los positivos. Según el plan de los empresarios del banco, un modelo más enfocado en el *recall* o en la *precision* puede ser más óptimo que el planteado acá.
  
También hay que destacar como los modelos entrenados con datos sobremuestrados presentaron una ligera ventaja sobre los del submuestreo. Eso se puede dar en que dentro de lo que Machine Learning respecta, 10000 no es un numero abismalmente grande de datos y posiblemente esas 4000 filas removidas en el proceso de submuestreo hayan repercutido de manera ligeramente notoria en las métricas de nuestros modelos.
  
Antes de irme quiero destacar que yo originalmente planeaba poner a prueba como se iba a comportar un modelo entrenado con los datos a los cuales simplemente se les eliminaron las filas con valores ausentes en `tenure` en vez de imputarlos con la media como hice yo. Mi hipótesis es que hubiesen sido ligeramente peor o igual ya que no eran ni 1000 datos faltantes por lo que nuestra imputación no tuvo tanto impacto en la distribución y por ende en los datos.